### INITIALIZATION

In [37]:
# Import external libraries:
import numpy as np
import skfem as sk
import scipy.sparse as sparse
import matplotlib.pyplot as plt

# Import external functions:
from skfem.helpers import dot, grad
from scipy.sparse.linalg import splu
from skfem.visuals.matplotlib import plot

In [39]:
# Define problem size:
length_0 = 000.0                # [cm] start loop point
length_H = 200.0                # [cm] core length
length_L = 150.0                # [cm] heat-exchanger length
length_T = length_H + length_L  # [cm] end loop point

# Define region identifier:
reg_H = lambda x: (x>=length_0) * (x<length_H)  # Core region identifier
reg_L = lambda x: (x>=length_H) * (x<length_T)  # Heat-exchanger identifier

# Define mesh discretization
mesh_width = 2.0                                            # Mesh element width [cm]
mesh_steps = int(length_T/mesh_width)                       # Number of mesh intervals
mash_nodes = np.linspace(length_0, length_T, mesh_steps+1)  # List of mesh nodes coordinates [cm]

# Define polinomial order for Galerkin discretization:
polynomial_order = 2

# Define continuous Galerkin discretization:
if polynomial_order==1 : line_element = sk.element.ElementLineP1()
if polynomial_order==2 : line_element = sk.element.ElementLineP2()
line_basis = sk.Basis(sk.MeshLine(mash_nodes), line_element)

In [57]:
# Define list of columns to keep in periodic domain configuration and reordering:
list_col = np.array(list(range(mesh_steps)) + list(range(mesh_steps+1, polynomial_order*mesh_steps+1)))
list_ord = np.hstack([[ii + jj*mesh_steps for jj in range(polynomial_order)] for ii in range(mesh_steps)])

# Convert continuous Galerkin matrix to periodic form:
def periodic_matrix(matrix):
    matrix[:, 0] += matrix[:, mesh_steps]
    matrix[0, :] += matrix[mesh_steps, :]
    return matrix[list_col, :][:, list_col]

# Define global diffusion bilinear form for continuous Galerkin:
@sk.BilinearForm
def diffusion_T(u, v, _): return dot(grad(u), grad(v))
KK_T = periodic_matrix(diffusion_T.assemble(line_basis))

# Define mass bilinear form on the full domain:
@sk.BilinearForm 
def mass_T(u, v, _): return u * v
MM_T = periodic_matrix(mass_T.assemble(line_basis))

# Define mass bilinear form on the core region:
@sk.BilinearForm
def mass_H(u, v, w): return reg_H(w.x[0]) * u * v
MM_H = periodic_matrix(mass_H.assemble(line_basis))

# Define zero matrix:
ZZ_T = sparse.csr_matrix(KK_T.shape)

In [61]:
# Neutron equation parameters (except cross sections and thermal coupling therms):
vel  = ...        #        Speed mono-energetic neutrons
beta = 5.4700e-3  # [1]    Delayed neutron fraction                     
nu   = 2.4355e+0  # [1]    Neutron yeld                                 
lam  = 8.0000e-2  # [s^-1] Decay constant of delayed neutron precursors
diff = 1.2100e0   # [cm]   Neutron diffusion coefficient at reference temperature (temp_0)

# Termal coupling parameters
alpha = 2.1200e-4  # [K^-1] Thermal expansion coefficient 
temp_0 = 9.2200e2  # [K] Reference temperature 

# Macroscopic cross sections:
sigma_f_0 = 4.4300e-3 / nu # [cm^-1] macroscopic fission cross section at reference temperature (temp_0)
sigma_a_0 = 3.7600e-3      # [cm^-1] macroscopic absorption cross section at reference temperature (temp_0)

# Define global reaction and diffusion coefficients:
k_nn_T =   vel * diff * (1 - alpha * temp_0)                        # Self diffusion coefficient on the whole domain
m_nn_T =   vel * sigma_a_0 * (1 + alpha * temp_0)                   # Self reaction coefficient on the whole domain
m_nc_T = - vel * lam                                                # Cross reaction coefficient from the precursors on the whole domain
m_nn_H = - vel * (1 - beta) * nu * sigma_f_0 * (1 + alpha * temp_0) # Self reaction coefficient on the core region only

# Define explicit non-linear terms coefficeints:
k_fn_T = - vel * diff * alpha                                      # Coefficient for the temperature neutron diffusion coupling (explicit)
m_fn_T =   vel * sigma_a_0 * alpha                                 # Coefficient for the temperature neutron absorption coupling (explicit)
m_fn_H = - vel * (1 - beta) * nu * sigma_f_0 * alpha               # Coefficient for the temperature neutron fission coupling (explicit)

# Assemble neutron equation left-hand side:
LHS_n = sparse.bmat([[k_nn_T*KK_T + m_nn_T*MM_T + m_nn_H*MM_H, m_nc_T*MM_T, ZZ_T]])